下面我们使用PyTorch来实现上一节中的多层感知机。首先导入所需的包或模块

In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l

c:\Users\Susan_Jay\.conda\envs\deeplearning\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\Susan_Jay\.conda\envs\deeplearning\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


# 3.10.1 定义模型
和softmax回归唯一的不同在于，我们多加了一个全连接层作为隐藏层。它的隐藏单元个数为256，并使用ReLU函数作为激活函数

In [2]:
# num_inputs = 784:
# 输入特征的数量
# 对应28×28=784像素的Fashion-MNIST图像展平后的向量长度
# num_outputs = 10
# 输出类别的数量
# 对应Fashion-MNIST的10个服装类别
# num_hiddens = 256:
# 隐藏层的神经元数量
# 决定了网络的表达能力
num_inputs, num_outputs, num_hiddens = 784, 10, 256


# nn.Sequential: 创建一个顺序容器，按顺序堆叠神经网络层
# d2l.FlattenLayer():
# 自定义的扁平化层，将多维输入展平为二维
# 例如：将(batch_size, 1, 28, 28)的张量转换为(batch_size, 784)

# nn.Linear(num_inputs, num_hiddens):
# 第一个全连接层
# 输入维度：784（每个像素一个特征）
# 输出维度：256（隐藏层神经元数量）
# 包含参数：权重W₁(256×784)和偏置b₁(256)

# nn.ReLU():
# ReLU激活函数：f(x) = max(0, x)
# 引入非线性变换，增强网络表达能力
# 帮助解决深度学习中的梯度消失问题

# nn.Linear(num_hiddens, num_outputs):
# 第二个全连接层
# 输入维度：256（隐藏层大小）
# 输出维度：10（类别数量）
# 包含参数：权重W₂(10×256)和偏置b₂(10)
net = nn.Sequential(
        d2l.FlattenLayer(),
        nn.Linear(num_inputs, num_hiddens),
        nn.ReLU(),
        nn.Linear(num_hiddens, num_outputs), 
        )

# net.parameters():
# 返回网络中所有可学习参数的迭代器
# 包括上述两个线性层的权重和偏置

# init.normal_():
# PyTorch的参数初始化函数
# 使用均值为0、标准差为0.01的正态分布初始化参数
# 下划线表示这是一个原地操作，直接修改传入的张量
# 良好的初始化有助于模型更快收敛
for params in net.parameters():
    init.normal_(params, mean=0, std=0.01)

# ---------------------------------------------------补充：---------------------------------------------------
# PyTorch 中线性层权重维度的说
# 数学上的理解 vs PyTorch 实现
# 从数学上理解线性变换：
# 如果输入是 784 维，输出是 256 维，那么变换矩阵 W 应该是 (256, 784)
# 因为: y = Wx，其中 x 是 (784, 1)，y 是 (256, 1)

# 在 PyTorch 的 nn.Linear 实现中：
# 对于 nn.Linear(in_features, out_features)
# 权重矩阵形状实际上就是 (out_features, in_features)
# 在我们的例子中，就是 (256, 784) 和 (10, 256)

# 3.10.2 读取数据并训练模型
我们使用与3.7节中训练softmax回归几乎相同的步骤来读取数据并训练模型。

注：由于这里使用的是PyTorch的SGD而不是d2lzh_pytorch里面的sgd，所以就不存在3.9节那样学习率看起来很大的问题了

In [3]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=0.5)

num_epochs = 5
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, None, None, optimizer)

epoch 1, loss 0.0031, train acc 0.706, test acc 0.787
epoch 2, loss 0.0019, train acc 0.820, test acc 0.792
epoch 3, loss 0.0016, train acc 0.844, test acc 0.840
epoch 4, loss 0.0015, train acc 0.857, test acc 0.825
epoch 5, loss 0.0014, train acc 0.862, test acc 0.846


# 小结
通过PyTorch可以更简洁地实现多层感知机